In [11]:
# PPO + GPT2, 中文情感分析

import time
import random

import torch
from rich import print
from tqdm import tqdm
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

from trl.gpt2 import GPT2HeadWithValueModel
from trl.ppo import PPOTrainer

from iTrainingLogger import iSummaryWriter

In [7]:
# !pip install rich

In [12]:
writer = iSummaryWriter(log_path = './logs', log_name = 'PPO-Sentiment-Zh')
config = {
    'model_name': 'uer/gpt2-chinese-cluecorpssmall',
    'steps': 10000,
    'batch_size': 32,
    'forward_batch_size': 16,
    'ppo_epoch': 4,
    'lr': 1.41e-5,
    'init_kl_ceof': 0.2,
    'target': 6,
    'horizon': 10000,
    'gamma': 1,
    'lam': 0.95,
    'cliprange': .2,
    'cliprange_value': .2,
    'vf_coef': .1,
    'gen_len': 16
}

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pipe_device = 0 if torch.cuda.is_available() else -1

In [14]:
prompts = [
    '刚收到货，感觉',
    '这部电影',
    '说实话，真的很',
    '这次购物总的来说体验很'
]

In [17]:
# 情感分类模型
senti_tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-jd-binary-chinese')
senti_model = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-jd-binary-chinese')
sentiment_pipe = pipeline('sentiment-analysis', model = senti_model, tokenizer = senti_tokenizer, device = pipe_device)

Downloading:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [18]:
# 文本生成模型
gpt2_model = GPT2HeadWithValueModel.from_pretrained(config['model_name'])
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(config['model_name'])
gpt2_tokenizer = AutoTokenizer.from_pretrained(config['model_name'])
gpt2_tokenizer.eos_token = gpt2_tokenizer.pad_token
gpt2_model.to(device)
gpt2_model_ref.to(device)

OSError: uer/gpt2-chinese-cluecorpssmall is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
gen_kwargs = {
    'min_length': -1,
    'top_k': 0.0,
    'top_p': 1.0,
    'do_sample': True,
    'pad_token_id': gpt2_tokenizer.eos_token_id
}